In [ ]:
'''
dont forgot to use the version tags in git after changing the code . we have to use 
those v tags once we are done with entire code and making some minor updates 
we need to push those tags separately.

'''

In [ ]:
# GPT link : https://chatgpt.com/c/67c31846-7cec-8003-9bae-1f349c1f37c1

tensorflow → Main framework for deep learning.

🔹 keras → High-level API inside TensorFlow for building neural networks.

🔹 Sequential → A linear stack of layers for CNN.

🔹 Conv2D, MaxPooling2D, Flatten, Dense, Dropout → Different layers for building CNN.

🔹 ImageDataGenerator → Helps load and preprocess images.

🔹 numpy → Used for numerical operations.

🔹 matplotlib.pyplot → Helps plot training results.

🔹 cv2 → OpenCV for image processing (resizing, conversion).

🔹 os → Helps in handling file paths.

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf  
from tensorflow import keras  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

import matplotlib.pyplot as plt
import os


in "tensorflow.keras.preprocessing.image", each word separated by a dot (.) represents a package, module, or submodule in Python.
tensorflow → The main TensorFlow package.

keras → A submodule inside TensorFlow that provides deep learning APIs.

preprocessing → A submodule inside Keras that contains tools for data preprocessing.

image → A module inside preprocessing that specifically deals with image data (e.g., ImageDataGenerator, load_img, img_to_array).



REASON FOR TARGET SIZE
every image in dataset is 512x512

Why Does 512x512 Require More GPU?
More Pixels → More Computation

A 512x512 image has 262,144 pixels, whereas a 224x224 image has only 50,176 pixels.
That’s 5x more pixels, meaning 5x more computation per image!


In [ ]:
# https://chatgpt.com/c/67d037a5-f280-8003-a91b-60bba6b2af07

# need to learn about mixed precision 

# Define ImageDataGenerator with Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=40,       # Rotate images randomly by 40 degrees
    width_shift_range=0.2,   # Shift width randomly
    height_shift_range=0.2,  # Shift height randomly
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Zoom in/out
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest',     # Fill missing pixels
    validation_split=0.2     # 80-20 train-validation split
)

# ImageDataGenerator for Validation (No Augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255,  
    validation_split=0.2
)




# Load Training Images with Augmentation
train_generator = train_datagen.flow_from_directory(
    "animals",  
    target_size=(224, 224),  
    batch_size=32,  
    class_mode="binary",  
    subset="training"
)

# Load Validation Images (No Augmentation)
validation_generator = val_datagen.flow_from_directory(
    "animals",  
    target_size=(224, 224),  
    batch_size=32,  
    class_mode="binary",  
    subset="validation"
)


Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [3]:
print(train_generator.class_indices)

{'cat': 0, 'dog': 1}
